In [32]:
##########################################################################
# 인스타 그램의 해쉬태그 수집하기 - by 서진수
##########################################################################
#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import os
import random
import unicodedata   # 인스타그램의 해시태그 수집 중 자음/모음 분리현상 방지용 모듈

#Step 2. 사용자에게 필요한 정보들을를 입력 받습니다.
print("=" *70)
print("   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다")
print("   본 제품은 서진수가 교육용으로 특별 제작했으며 ")
print("   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다")
print("   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^")
print("=" *70)

v_id = input("1.인스타그램의 ID를 입력하세요: ")
v_passwd = input("2.인스타그램의 비밀번호를 입력하세요: ")
query_txt = input("3.검색할 해쉬태그를 입력하세요(예: 강남맛집): ")
cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
real_cnt = math.ceil(cnt / 10)

f_dir=input('5.파일이 저장될 경로만 쓰세요(기본경로 : c:\\temp\\ ) : ')
if f_dir =='' :
    f_dir = "c:\\temp\\"

#Step 3. 결과를 저장할 폴더명과 파일명을 설정하고 폴더를 생성합니다.
s_time = time.time( )
query_txt2 = '인스타그램'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

# Step 4. 인스타그램 자동 로그인 하기
chrome_path = "c:/temp/chromedriver_85/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get("https://www.instagram.com/")
time.sleep(random.randrange(1,5))

print("\n")
print("요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^")
print("\n")

#ID와 비번 입력후 로그인하기
eid = driver.find_element_by_name('username')
for a in v_id :
        eid.send_keys(a)
        time.sleep(0.3)
epwd = driver.find_element_by_name('password')
for b in v_passwd :
        epwd.send_keys(b)
        time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[3]/button/div').click()
time.sleep(5)

# Step 5. 검색할 해쉬태그 입력하기
#로그인 정보 나중에 저장하기
driver.find_element_by_xpath('//*[@id="react-root"]/section/main/div/div/div/div/button').click()
time.sleep(2)

driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()
time.sleep(1)

# 검색할 키워드 입력하기
element = driver.find_element_by_xpath('''//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input''')

for c in query_txt :
    element.send_keys(c)
    time.sleep(0.2)

time.sleep(5)
driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]/div').click()

import pandas as pd

# 자동 스크롤다운 함수
def scroll_down(driver):
  driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
  time.sleep(5)

print('요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^')
item=[]     # 인스타그램 URL 주소 저장할 리스트
item2=[]
    
a = 1
while (a <= real_cnt):
    scroll_down(driver) 

    # Step 6. 전체 게시물의 원본 URL 추출하기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    all = soup.find('article','KC1QD').find_all('a')

    for i in all:    
        url = i['href']
        item.append(url)          
        item2 = pd.Series(item).drop_duplicates()
    
        if len(item2) >= cnt :
            break
    a += 1
    print('요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^')
    
    if len(item2) >= cnt :
        break
   
# 추출된 URL 사용하여 전체 URL 완성하기
full_url=[]
url_cnt = 0
for x in range(0,len(item2)) :
    url = 'https://www.instagram.com' +item[x]
    full_url.append(url)
    url_cnt += 1
    
    if url_cnt > cnt:
        break

# 아래 for 반복문은 수집된 URL 주소를 확인하는 코드임        
#for c in range(1,len(full_url)+1) :
#    print(c,':',full_url[c-1])

#Step 7. 각 페이지별로 그림과 해쉬태그를 수집하기
count = 1        # 추출 데이터 건수 세기
hash_txt = []    # 해쉬 태그 저장 

# 비트맵 이미지 아이콘을 위한 대체 딕셔너리를 만든다
import sys
bmp_map = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 0xfffd)

count = 0

os.makedirs(f_dir+s+'-'+query_txt2+'-'+query_txt)
f_name=f_dir+s+'-'+query_txt2+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'

for c in range(0,len(full_url)) :
    driver.get(full_url[c])
    time.sleep(2)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    f = open(f_name, 'a',encoding='UTF-8')

    # 해당 페이지의 해시태그 수집
    tags = soup.find('div','EtaWk')

    try :
        tags_1 = tags.find_all('a')
    except :
        pass
    else :
        for d in range(0, len(tags_1)) :
            tags = tags_1[d].get_text()
            tags_11 = tags.translate(bmp_map)
            tags_2 = unicodedata.normalize('NFC', tags_11)

            for i in tags_2 :
                if i[0:1]=='#' :
                    hash_txt.append(tags_2)
                    print(tags_2)
                    f.write("\n" + str(tags_2))
    f.close()
    count += 1
    
#Step 7. 요약 정보 출력하기    
e_time = time.time( )
t_time = e_time - s_time

print("=" *100)
print("총 소요시간: %s 초" %round(t_time,1))
print("총 저장 건수: %s 건 " %count)
print("파일 저장 경로: %s" %f_name)
print("=" *100)

driver.close( )

   이 크롤러는 인스타그램의 해시태그 정보를 수집합니다
   본 제품은 서진수가 교육용으로 특별 제작했으며 
   용도외의 사용으로 저작권을 침해하는 행위는 불법입니다
   본 제품에 대한 문의는 seojinsu@gmail.com 으로 보내주세요~^^
1.인스타그램의 ID를 입력하세요: gachilabs
2.인스타그램의 비밀번호를 입력하세요: gachilabs123
3.검색할 해쉬태그를 입력하세요(예: 강남맛집): 강남역맛집
4.크롤링 할 건수는 몇건입니까?: 40
5.파일이 저장될 경로만 쓰세요(기본경로 : c:\temp\ ) : 


요청하신 데이터를 추출중이오니 잠시만 기다려 주세요~~~~^^


요청하신 데이터를 수집중이니 잠시만 기다려 주세요~^^
요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^
요청하신 데이터를 수집중이니 잠시만 더 기다려 주세요~^^
1 : https://www.instagram.com/p/CG_czHjloII/
2 : https://www.instagram.com/p/CHEwmG0nshR/
3 : https://www.instagram.com/p/CHFQh7AHiqn/
4 : https://www.instagram.com/p/CG9HLgoh5QP/
5 : https://www.instagram.com/p/CG_fUOcHFOz/
6 : https://www.instagram.com/p/CHHEDeSjvnA/
7 : https://www.instagram.com/p/CGiyR8NhQ5P/
8 : https://www.instagram.com/p/CHFQCJLJSM-/
9 : https://www.instagram.com/p/CHFpgzPnBbH/
10 : https://www.instagram.com/p/CHHkQ7jl8Eo/
11 : https://www.instagram.com/p/CHHkAwTlu9v/
12 : https://www.instagram.com/p/CHHjizzF4A_/
13 : https://www.i